In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np


In [17]:
import os
import glob
import json

In [4]:
os.chdir('/home/walml/repos/recommender_hack')

In [65]:
power['comment_user_id']

0             54
1        1428920
2           3870
3        1337311
4        1373986
          ...   
68444      31153
68445      31153
68446      31153
68447    1832099
68448      31153
Name: comment_user_id, Length: 68449, dtype: int64

In [79]:
power['comment_user_login']

0                  klmasters
1                    MerylPG
2        trevor_allen_faller
3                   discala_
4            Ghost_Sheep_SWR
                ...         
68444                    ine
68445                    ine
68446                    ine
68447          Enceladusmoon
68448                    ine
Name: comment_user_login, Length: 68449, dtype: object

In [48]:
power = pd.read_csv('power.csv')
users_on_talk = set(power['comment_user_id'].unique())  # faster lookup in sets

In [49]:
flat_dir = '/media/walml/beta/decals/results/streaming/flat'

In [50]:
assert os.path.isdir(flat_dir)

In [51]:
paths = glob.glob(flat_dir + '/*.json')

In [52]:
path = paths[0]

In [53]:
from collections import namedtuple

In [54]:
Pair = namedtuple('Pair', ['user', 'galaxy'])

In [95]:
def load_pairs(path, users):
    pairs = []
    with open(path, 'r') as f:
        while True:
            line = f.readline()
            if line == '':
                break
            data = json.loads(line)
            try:
                if str(data['user_id']) in users_on_talk:
                    pairs.append(Pair(str(data['user_id']), str(data['subject_id'])))
            except KeyError:
                pass  # no user id = not logged in
    return pairs

In [96]:
# users_on_talk

In [97]:
pairs = set(load_pairs(path, users_on_talk))  # set for unique

In [98]:
len(pairs)

239

In [99]:
len(paths)

1129

In [100]:
all_pairs = []
for path in paths:
    all_pairs = all_pairs + load_pairs(path, users_on_talk)

In [101]:
all_pairs = set(all_pairs)
len(all_pairs)

406395

In [102]:
pair_df = pd.DataFrame(data=all_pairs)
pair_df.head()

,user,galaxy
0,142382,36254885
1,1807448,25809637
2,1361053,27361502
3,1392258,35622163
4,1856537,28481842


In [103]:
pair_df['user_id'] = pair_df['user']
del pair_df['user']
pair_df['subject_id'] = pair_df['galaxy']
del pair_df['galaxy']
pair_df.head()

,user_id,subject_id
0,142382,36254885
1,1807448,25809637
2,1361053,27361502
3,1392258,35622163
4,1856537,28481842


In [104]:
pair_df.to_csv('../pair_df.csv', index=False)